

## MTCNN MODEL 발췌

In [ ]:
from facenet_pytorch import MTCNN
import torch

In [ ]:
# 장치 설정 (GPU 사용 가능 시 GPU로 설정)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# FastMTCNN 클래스 정의
class FastMTCNN(object):
    """Fast MTCNN implementation."""

    def __init__(self, stride, resize=1, *args, **kwargs):
        """Constructor for FastMTCNN class.

        Arguments:
            stride (int): The detection stride. Faces will be detected every `stride` frames
                and remembered for `stride-1` frames.

        Keyword arguments:
            resize (float): Fractional frame scaling. [default: {1}]
            *args: Arguments to pass to the MTCNN constructor. See help(MTCNN).
            **kwargs: Keyword arguments to pass to the MTCNN constructor. See help(MTCNN).
        """
        self.stride = stride
        self.resize = resize
        self.mtcnn = MTCNN(*args, **kwargs)

    def __call__(self, frames):
        """Detect faces in frames using strided MTCNN."""
        if self.resize != 1:
            frames = [
                cv2.resize(f, (int(f.shape[1] * self.resize), int(f.shape[0] * self.resize)))
                    for f in frames
            ]

        boxes, probs = self.mtcnn.detect(frames[::self.stride])
        faces = []
        for i, frame in enumerate(frames):
            box_ind = int(i / self.stride)
            if boxes[box_ind] is None:
                continue
            for box in boxes[box_ind]:
                box = [int(b) for b in box]
                faces.append(frame[box[1]:box[3], box[0]:box[2]])
        return faces

In [ ]:
# FastMTCNN 객체 생성
fast_mtcnn = FastMTCNN(
    stride=4,
    resize=0.5,
    margin=50,
    factor=0.6,
    keep_all=True,
    device=device,
    min_face_size=60
)